**Recommend from Top 100 and from Spotify clusters**

In [2]:
import pandas as pd
import random
import numpy as np
from config import *
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pickle

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID, client_secret=Client_Secret))

In [4]:
top100_df = pd.read_csv("../data/top100_df.csv")
spotify_df = pd.read_csv("../data/clusters_df.csv")

In [5]:
top100_df = top100_df.apply(lambda x: x.astype(str).str.lower()) #turning dataframes to lower case
spotify_df = spotify_df.apply(lambda x: x.astype(str).str.lower())

In [6]:
def song_recommendation():

    while True:
        user_input = input("What is the name of your favourite song? \n")
        user_input = user_input.lower()
        
        if user_input in top100_df.values:
            top100_df.drop(top100_df.loc[top100_df['song_title'] == user_input].index, inplace=True)
            song_choice = random.choice(top100_df['song_title'])
            suggestion = top100_df.loc[top100_df['song_title'] == song_choice]
            artist = (suggestion.loc[suggestion.song_title == song_choice, 'artist'].values[0])
            print("\nMaybe you will like:", song_choice.title(),
                  " by ", artist.title())
            print("\nWe hope you like our recommendation!")
            
            ask_again = input("Would you like another recommendation? Please type 'yes' ")
            ask_again = ask_again.lower()
            print("")
            
            if ask_again == "yes":
                continue
            else:
                print("Thanks for stopping by - see you soon!")
                break
            break
        
        else:
            artist_input = input("Who is the artist of this song?\n")
            results = sp.search(q="track:"+user_input+" artist:"+artist_input,limit=10)
            print("searching for", results['tracks']['items'][0]['name'], "by", results['tracks']['items'][0]['artists'][0]['name'])
            features = sp.audio_features(results['tracks']['items'][0]['uri'])
            features_df = pd.DataFrame(features)
            features_df = features_df[['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']]
            with open("scaler.pickle", "rb") as f: 
                scaler =  pickle.load(f)
            features_array = scaler.transform(features_df)
            features_df_transformed = pd.DataFrame(features_array, columns = features_df.columns)
            with open("kmeans.pickle", "rb") as f:
                kmeans = pickle.load(f)
            
            cluster_song = kmeans.predict(features_df_transformed)[0]
            ##print(cluster_song)
            
            df_float = spotify_df[spotify_df['cluster'] == str(cluster_song)]
            song_choice = random.choice(df_float['song_name'].tolist())
            suggestion = df_float.loc[spotify_df['song_name'] == song_choice]
            artist = (suggestion.loc[suggestion.song_name == song_choice, 'artist_name'].values[0])
            print("\nMaybe you will like:", song_choice.title(),
                  " by ", artist.title())
            print("\nWe hope you like our recommendation!\n")
            
            ask_again = input("Would you like another recommendation? Please type 'yes' ")
            ask_again = ask_again.lower()
            print("")
            
            if ask_again == "yes":
                continue
            else:
                print("Thanks for stopping by - see you soon!")
                break

In [7]:
song_recommendation()

What is the name of your favourite song? 
holocene
Who is the artist of this song?
bon iver
searching for Holocene by Bon Iver

Maybe you will like: Blue  by  Joni Mitchell 

We hope you like our recommendation!

Would you like another recommendation? Please type 'yes' yes

What is the name of your favourite song? 
peaches

Maybe you will like: Blame On You  by  Rod Wave

We hope you like our recommendation!


KeyboardInterrupt: Interrupted by user